In [171]:
# Request를 이용한 정적 크롤링 수행해보기
from urllib import request

# 오펜하이머 영화 정보 및 리뷰가 있는 IMDb라는 사이트
url = 'https://www.imdb.com/title/tt15398776/reviews?ref_=tt_urv'
html = request.urlopen(url).read().decode('utf8')
html[:60]

'\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/'

In [172]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

reviews=[]
#review 내용 가져오기
contents = soup.find_all('div',attrs={'class':'review-container'})
for content in contents:
  txt=content.div.text
  #텍스트 정규화 movie -> film
  txt=txt.replace("movie", "film")
  reviews.append(txt)
  #print(reviews[-1])
  #print('-'*200)

In [173]:
import re

cnt=0
ratings=[]
for i,review in enumerate(reviews[:]):
  temp=re.findall('[0-9]+/[0-9]+', review)

  #평점이 없는 경우 몇개 있어서 삭제
  if len(temp)==0:
    cnt+=1
    del reviews[i-cnt]
    continue
  #후기내부에 5/10같은 형태의 내용이 있어 제일 처음발견한 것을 후기로 사용
  ratings.append(temp[0])
ratings

['9/10',
 '8/10',
 '10/10',
 '10/10',
 '8/10',
 '10/10',
 '10/10',
 '10/10',
 '9/10',
 '6/10',
 '8/10',
 '6/10',
 '5/10',
 '8/10',
 '8/10',
 '9/10',
 '10/10',
 '8/10',
 '8/10',
 '7/10',
 '10/10',
 '7/10',
 '4/10']

In [174]:
#Stemming
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
print(reviews[0])
for i,review in enumerate(reviews):
  reviews[i] = " ".join([stemmer.stem(word) for word in word_tokenize(review)])

print(reviews[0])









9/10


 Murphy is exceptional
 
Orlando_Gardner19 July 2023


You'll have to have your wits about you and your brain fully switched on watching Oppenheimer as it could easily get away from a nonattentive viewer. This is intelligent filmmaking which shows it's audience great respect. It fires dialogue packed with information at a relentless pace and jumps to very different times in Oppenheimer's life continuously through it's 3 hour runtime. There are visual clues to guide the viewer through these times but again you'll have to get to grips with these quite quickly. This relentlessness helps to express the urgency with which the US attacked it's chase for the atomic bomb before Germany could do the same. An absolute career best performance from (the consistenly brilliant) Cillian Murphy anchors the film. This is a nailed on Oscar performance. In fact the whole cast are fantastic (apart maybe for the sometimes overwrought Emily Blunt performance). RDJ is also particularly brillia

In [177]:
#정규식으로 리뷰가 아닌 아래 3문장 지우기
# 몇명 중에 몇명이 리뷰가 도움이 되었다고 했습니다.
# 이 리뷰가 도움이 됐나요?
# Permalink
import nltk
from nltk import sent_tokenize
nltk.download('punkt')

text=[]
for review in reviews:
  text.append([s for s in sent_tokenize(review) if not (re.match(r'\d+,\d+ out of \d+,\d+ found thi help .', s) or re.match(r'wa thi review help ?', s) or re.match(r'sign in to vote .', s) or re.match(r'permalink', s)) ])
text[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["9/10 murphi is except orlando_gardner19 juli 2023 you 'll have to have your wit about you and your brain fulli switch on watch oppenheim as it could easili get away from a nonattent viewer .",
 "thi is intellig filmmak which show it 's audienc great respect .",
 "it fire dialogu pack with inform at a relentless pace and jump to veri differ time in oppenheim 's life continu through it 's 3 hour runtim .",
 "there are visual clue to guid the viewer through these time but again you 'll have to get to grip with these quit quickli .",
 "thi relentless help to express the urgenc with which the us attack it 's chase for the atom bomb befor germani could do the same .",
 'an absolut career best perform from ( the consistenli brilliant ) cillian murphi anchor the film .',
 'thi is a nail on oscar perform .',
 'in fact the whole cast are fantast ( apart mayb for the sometim overwrought emili blunt perform ) .',
 'rdj is also particularli brilliant in a return to proper act after hi decad or so

In [179]:
all_text=''

for t in text:
  for s in t:
    all_text+=s

all_text

"9/10 murphi is except orlando_gardner19 juli 2023 you 'll have to have your wit about you and your brain fulli switch on watch oppenheim as it could easili get away from a nonattent viewer .thi is intellig filmmak which show it 's audienc great respect .it fire dialogu pack with inform at a relentless pace and jump to veri differ time in oppenheim 's life continu through it 's 3 hour runtim .there are visual clue to guid the viewer through these time but again you 'll have to get to grip with these quit quickli .thi relentless help to express the urgenc with which the us attack it 's chase for the atom bomb befor germani could do the same .an absolut career best perform from ( the consistenli brilliant ) cillian murphi anchor the film .thi is a nail on oscar perform .in fact the whole cast are fantast ( apart mayb for the sometim overwrought emili blunt perform ) .rdj is also particularli brilliant in a return to proper act after hi decad or so of call it in .the screenplay is dens an

In [186]:
import nltk
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist

nltk.download('punkt')
nltk.download('stopwords')
stop_words=stopwords.words('english')

def split_sentences(sent):

  #모든 리뷰에서 FreaDist계산
  fdist=FreqDist(word_tokenize(all_text.lower()))

  sents = sent_tokenize(sent)
  output=[]
  for sent in sents:
    words=[]
    for w in word_tokenize(sent.lower()):
      if w.lower() not in stop_words and len(w)>=4 and fdist[w]>=5:
        words.append(w)
    output.append(words)
  return output

for i,t in enumerate(text):
  print(ratings[i])
  print(split_sentences(' '.join(t)))
  print('-'*300)
  print()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


9/10
[['murphi', 'except', 'juli', '2023', 'watch', 'oppenheim', 'could', 'viewer'], ['filmmak', 'show', 'audienc', 'great'], ['pace', 'veri', 'differ', 'time', 'oppenheim', 'life', 'hour', 'runtim'], ['visual', 'viewer', 'time', 'quit'], ['help', 'atom', 'bomb', 'befor', 'could'], ['best', 'perform', 'brilliant', 'cillian', 'murphi', 'film'], ['oscar', 'perform'], ['whole', 'cast', 'mayb', 'sometim', 'emili', 'blunt', 'perform'], ['also', 'brilliant'], ['cinematographi', 'quit', 'part', 'moment', 'especi', 'scene', 'pugh', 'score', 'beauti', 'time', 'pace'], ['hour', 'runtim'], ['found', 'watch'], ['film', 'make'], ['realli', 'great', 'watch']]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

8/10
[['watch', 'sure'], ['juli', '2023', 'f